<a href="https://colab.research.google.com/github/QiaoYe-Learning/NLP-classwork/blob/main/%E2%80%9CAssignment_1_Simple_Sentiment_Analysis_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download dataset
!pip install torchtext==0.4.0
# import
import torch
from torchtext import data
from torchtext import datasets # pre-define dataset supported by torchtext
import random
import torch.nn as nn # Neural Network
import time
import torch.optim as optim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.9 MB/s eta 0:00:00


In [2]:
# Model Building And Function Defining
class RNN(nn.Module): # inherited from nn.Module
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim): # vocabulary size, embedding_dim, hidden_dim, classification number

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        #text = [sentence length, batch size], text_lengths: real length

        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim] - The hidden status of each time step, containing information about all time steps.
        #hidden = [1, batch size, hid dim] - The hidden state of the last time step

        #assert torch.equal(output[-1,:,:], hidden.squeeze(0)) - Verify that the output of the last time step is equal to the hidden state

        return self.fc(hidden.squeeze(0))

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train() # set model - train

    for batch in iterator:

        optimizer.zero_grad() # Clear the gradient in the optimizer before each forward propagation

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward() # Calculate the gradient of the loss by back propagation

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval() # setting model - evaluate

    with torch.no_grad(): # No back propagation is required during the evaluation process, so the gradient calculation is turned off

        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [3]:
# Initial setting
SEED = 1234 # random seed
torch.manual_seed(SEED) # normalize random seed

torch.backends.cudnn.deterministic = True # enture CUDA's behavior is stable（determinacy）

TEXT = data.Field(tokenize='spacy', # split word using spacy
                  tokenizer_language = 'en_core_web_sm', # enture spacy using English_model
                  include_lengths = True, # include the length of sentence when return the data
                  pad_first=True) # add a mark before the sequence
LABEL = data.LabelField(dtype = torch.float) # specfies the tyoe of label data is 'float'(binary classfication and regression)

training_data, test_data = datasets.IMDB.splits(TEXT, LABEL) # load dataset-IMDB(binary classfication task)
train_data, valid_data = training_data.split(random_state = random.seed(SEED)) # Default 70/30
# train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED)) 80/5 = train/valid


# Build a vocabulary
MAX_VOCAB_SIZE = 25_000 # max size of vocabulary, only top 25000, other is <unk>
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE) # build train vocabulary
LABEL.build_vocab(train_data) # build vicabulary for the label fields <unk>,<pad>


# Using BucketIterator to create data iterator for train, valid, test to processes data in batch
BATCH_SIZE = 64 # size of samples per batch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # device choice, preferential using GPU(cuda)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True, # sentences in each batch are sorted by their length, useful to reduce the using of <pad>
    device = device)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:23<00:00, 3.52MB/s]


In [4]:
# Model Settings
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model_RNN_main = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [5]:
# Main-RNN
N_EPOCHS = 20
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task
model_RNN = model_RNN_main.to(device)
criterion = criterion.to(device)

# Optimizer Settings
optimizer_SGD = optim.SGD(model_RNN_main.parameters(), lr=1e-3) # SGD optimizer - Stochastic gradient descent
optimizer_Adam = optim.Adam(model_RNN_main.parameters(), lr=1e-3) # Adam optimizer - Stochastic gradient descent
optimizer_Adagrad = optim.Adagrad(model_RNN_main.parameters(), lr=1e-3) # Adagrad optimizer - Stochastic gradient descent

In [7]:
## SGD
best_valid_loss_SGD_20 = float('inf')
epoch=0
for epoch in range(N_EPOCHS):
    start_time_SGD_20 = time.time()

    train_loss_SGD_20, train_acc_SGD_20 = train(model_RNN, train_iterator, optimizer_SGD, criterion)
    valid_loss_SGD_20, valid_acc_SGD_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_SGD_20 = time.time()

    epoch_mins_SGD_20, epoch_secs_SGD_20 = epoch_time(start_time_SGD_20, end_time_SGD_20)

    if valid_loss_SGD_20 < best_valid_loss_SGD_20:
        best_valid_loss = valid_loss_SGD_20
        torch.save(model_RNN.state_dict(), 'tut1-model_SGD_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_SGD_20}m {epoch_secs_SGD_20}s')
    print(f'\tTrain Loss-SGD_20: {train_loss_SGD_20:.3f} | Train Acc-SGD_20: {train_acc_SGD_20*100:.2f}%')
    print(f'\t Val. Loss-SGD_20: {valid_loss_SGD_20:.3f} |  Val. Acc-SGD_20: {valid_acc_SGD_20*100:.2f}%')

# SGD_20
model_RNN.load_state_dict(torch.load('tut1-model_SGD_20.pt'))
test_loss_SGD_20, test_acc_SGD_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_SGD_20: {test_loss_SGD_20:.3f} | Test Acc_SGD_20: {test_acc_SGD_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss-SGD_20: 0.693 | Train Acc-SGD_20: 51.93%
	 Val. Loss-SGD_20: 0.693 |  Val. Acc-SGD_20: 51.86%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss-SGD_20: 0.691 | Train Acc-SGD_20: 52.26%
	 Val. Loss-SGD_20: 0.692 |  Val. Acc-SGD_20: 51.81%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.690 | Train Acc-SGD_20: 52.63%
	 Val. Loss-SGD_20: 0.692 |  Val. Acc-SGD_20: 51.55%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.690 | Train Acc-SGD_20: 52.70%
	 Val. Loss-SGD_20: 0.691 |  Val. Acc-SGD_20: 51.88%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss-SGD_20: 0.689 | Train Acc-SGD_20: 52.92%
	 Val. Loss-SGD_20: 0.690 |  Val. Acc-SGD_20: 52.41%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.688 | Train Acc-SGD_20: 53.15%
	 Val. Loss-SGD_20: 0.690 |  Val. Acc-SGD_20: 52.51%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.688 | Train Acc-SGD_20: 53.41%
	 Val. Loss-SGD_20: 0.689 |  Val. Acc-SGD_20: 53.24%
Epoch: 08 | Epoch Time: 0m 4s
	Train Loss-SGD_20

<ipython-input-7-8b315b528752>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_SGD_20.pt'))


Test Loss_SGD_20: 0.684 | Test Acc_SGD_20: 55.45%


In [8]:
## Adam_20
best_valid_loss_Adam_20 = float('inf')
epoch=0
for epoch in range(N_EPOCHS):
    start_time_Adam_20 = time.time()

    train_loss_Adam_20, train_acc_Adam_20 = train(model_RNN, train_iterator, optimizer_Adam, criterion)
    valid_loss_Adam_20, valid_acc_Adam_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adam_20 = time.time()

    epoch_mins_Adam_20, epoch_secs_Adam_20 = epoch_time(start_time_Adam_20, end_time_Adam_20)

    if valid_loss_Adam_20 < best_valid_loss_Adam_20:
        best_valid_loss = valid_loss_Adam_20
        torch.save(model_RNN.state_dict(), 'tut1-model_Adam_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adam_20}m {epoch_secs_Adam_20}s')
    print(f'\tTrain Loss-Adam_20: {train_loss_Adam_20:.3f} | Train Acc-Adam_20: {train_acc_Adam_20*100:.2f}%')
    print(f'\t Val. Loss-Adam_20: {valid_loss_Adam_20:.3f} |  Val. Acc-Adam_20: {valid_acc_Adam_20*100:.2f}%')

# Adam_20
model_RNN.load_state_dict(torch.load('tut1-model_Adam_20.pt'))
test_loss_Adam_20, test_acc_Adam_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adam_20: {test_loss_Adam_20:.3f} | Test Acc_Adam_20: {test_acc_Adam_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.677 | Train Acc-Adam_20: 57.53%
	 Val. Loss-Adam_20: 0.710 |  Val. Acc-Adam_20: 48.16%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.644 | Train Acc-Adam_20: 62.35%
	 Val. Loss-Adam_20: 0.614 |  Val. Acc-Adam_20: 67.10%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.575 | Train Acc-Adam_20: 69.85%
	 Val. Loss-Adam_20: 0.579 |  Val. Acc-Adam_20: 71.28%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.536 | Train Acc-Adam_20: 73.79%
	 Val. Loss-Adam_20: 0.575 |  Val. Acc-Adam_20: 72.33%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.479 | Train Acc-Adam_20: 77.79%
	 Val. Loss-Adam_20: 0.576 |  Val. Acc-Adam_20: 71.56%
Epoch: 06 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.546 | Train Acc-Adam_20: 71.43%
	 Val. Loss-Adam_20: 0.667 |  Val. Acc-Adam_20: 60.40%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.656 | Train Acc-Adam_20: 58.66%
	 Val. Loss-Adam_20: 0.693 |  Val. Acc-Adam_20: 51.43%
Epoch: 08 | Epoch Ti

<ipython-input-8-42a6fdf30cbd>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adam_20.pt'))


Test Loss_Adam_20: 0.693 | Test Acc_Adam_20: 73.51%


In [12]:
## Adagrad_20
best_valid_loss_Adagrad_20 = float('inf')
epoch = 0
for epoch in range(N_EPOCHS):
    start_time_Adagrad_20 = time.time()

    train_loss_Adagrad_20, train_acc_Adagrad_20 = train(model_RNN, train_iterator, optimizer_Adagrad, criterion)
    valid_loss_Adagrad_20, valid_acc_Adagrad_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adagrad_20 = time.time()

    epoch_mins_Adagrad_20, epoch_secs_Adagrad_20 = epoch_time(start_time_Adagrad_20, end_time_Adagrad_20)

    if valid_loss_Adagrad_20 < best_valid_loss_Adagrad_20:
        best_valid_loss_Adagrad_20 = valid_loss_Adagrad_20
        torch.save(model_RNN.state_dict(), 'tut1-model_Adagrad_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adagrad_20}m {epoch_secs_Adagrad_20}s')
    print(f'\tTrain Loss-Adagrad_20: {train_loss_Adagrad_20:.3f} | Train Acc-Adagrad_20: {train_acc_Adagrad_20*100:.2f}%')
    print(f'\t Val. Loss-Adagrad_20: {valid_loss_Adagrad_20:.3f} |  Val. Acc-Adagrad_20: {valid_acc_Adagrad_20*100:.2f}%')

# Adagrad_20
model_RNN.load_state_dict(torch.load('tut1-model_Adagrad_20.pt'))
test_loss_Adagrad_20, test_acc_Adagrad_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adagrad_20: {test_loss_Adagrad_20:.3f} | Test Acc_Adagrad_20: {test_acc_Adagrad_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss-Adagrad_20: 0.683 | Train Acc-Adagrad_20: 55.33%
	 Val. Loss-Adagrad_20: 0.675 |  Val. Acc-Adagrad_20: 57.30%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.645 | Train Acc-Adagrad_20: 62.39%
	 Val. Loss-Adagrad_20: 0.638 |  Val. Acc-Adagrad_20: 63.41%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.641 | Train Acc-Adagrad_20: 63.44%
	 Val. Loss-Adagrad_20: 0.665 |  Val. Acc-Adagrad_20: 59.12%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss-Adagrad_20: 0.642 | Train Acc-Adagrad_20: 63.58%
	 Val. Loss-Adagrad_20: 0.651 |  Val. Acc-Adagrad_20: 61.70%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.629 | Train Acc-Adagrad_20: 64.42%
	 Val. Loss-Adagrad_20: 0.638 |  Val. Acc-Adagrad_20: 63.40%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.619 | Train Acc-Adagrad_20: 65.57%
	 Val. Loss-Adagrad_20: 0.631 |  Val. Acc-Adagrad_20: 64.43%
Epoch: 07 | Epoch Time: 0m 4s
	Train Loss-Adagrad_20: 0.611 | Train Acc-Adagrad_20: 66.3

<ipython-input-12-9acf827037f4>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adagrad_20.pt'))


Test Loss_Adagrad_20: 0.596 | Test Acc_Adagrad_20: 68.62%


In [34]:
## Adam_5
# Model Settings
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model_RNN_main = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task
model_RNN = model_RNN_main.to(device)
criterion = criterion.to(device)

best_valid_loss_Adam_5 = float('inf')
epoch=0
N_EPOCHS=5
optimizer_Adam = optim.Adam(model_RNN_main.parameters(), lr=1e-3) # Adam optimizer - Stochastic gradient descent
for epoch in range(N_EPOCHS):
    start_time_Adam_5 = time.time()

    train_loss_Adam_5, train_acc_Adam_5 = train(model_RNN, train_iterator, optimizer_Adam, criterion)
    valid_loss_Adam_5, valid_acc_Adam_5 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adam_5 = time.time()

    epoch_mins_Adam_5, epoch_secs_Adam_5 = epoch_time(start_time_Adam_5, end_time_Adam_5)

    if valid_loss_Adam_5 < best_valid_loss_Adam_5:
        best_valid_loss = valid_loss_Adam_5
        torch.save(model_RNN.state_dict(), 'tut1-model_Adam_5.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adam_5}m {epoch_secs_Adam_5}s')
    print(f'\tTrain Loss-Adam_5: {train_loss_Adam_5:.3f} | Train Acc-Adam_5: {train_acc_Adam_5*100:.2f}%')
    print(f'\t Val. Loss-Adam_5: {valid_loss_Adam_5:.3f} |  Val. Acc-Adam_5: {valid_acc_Adam_5*100:.2f}%')

# Adam_5
model_RNN.load_state_dict(torch.load('tut1-model_Adam_5.pt'))
test_loss_Adam_5, test_acc_Adam_5 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adam_5: {test_loss_Adam_5:.3f} | Test Acc_Adam_5: {test_acc_Adam_5*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss-Adam_5: 0.685 | Train Acc-Adam_5: 55.48%
	 Val. Loss-Adam_5: 0.669 |  Val. Acc-Adam_5: 58.32%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss-Adam_5: 0.653 | Train Acc-Adam_5: 61.52%
	 Val. Loss-Adam_5: 0.687 |  Val. Acc-Adam_5: 60.39%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss-Adam_5: 0.621 | Train Acc-Adam_5: 65.04%
	 Val. Loss-Adam_5: 0.672 |  Val. Acc-Adam_5: 57.00%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-Adam_5: 0.662 | Train Acc-Adam_5: 58.14%
	 Val. Loss-Adam_5: 0.693 |  Val. Acc-Adam_5: 51.65%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adam_5: 0.693 | Train Acc-Adam_5: 52.58%
	 Val. Loss-Adam_5: 0.691 |  Val. Acc-Adam_5: 53.84%


<ipython-input-34-8d037276dc0c>:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adam_5.pt'))


Test Loss_Adam_5: 0.692 | Test Acc_Adam_5: 53.55%


In [35]:
## Adam_10
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model_RNN_main = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task
model_RNN = model_RNN_main.to(device)
criterion = criterion.to(device)

best_valid_loss_Adam_10 = float('inf')
epoch=0
N_EPOCHS=10
optimizer_Adam = optim.Adam(model_RNN_main.parameters(), lr=1e-3) # Adam optimizer - Stochastic gradient descent
for epoch in range(N_EPOCHS):
    start_time_Adam_10 = time.time()

    train_loss_Adam_10, train_acc_Adam_10 = train(model_RNN, train_iterator, optimizer_Adam, criterion)
    valid_loss_Adam_10, valid_acc_Adam_10 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adam_10 = time.time()

    epoch_mins_Adam_10, epoch_secs_Adam_10 = epoch_time(start_time_Adam_10, end_time_Adam_10)

    if valid_loss_Adam_10 < best_valid_loss_Adam_10:
        best_valid_loss = valid_loss_Adam_10
        torch.save(model_RNN.state_dict(), 'tut1-model_Adam_10.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adam_10}m {epoch_secs_Adam_10}s')
    print(f'\tTrain Loss-Adam_10: {train_loss_Adam_10:.3f} | Train Acc-Adam_10: {train_acc_Adam_10*100:.2f}%')
    print(f'\t Val. Loss-Adam_10: {valid_loss_Adam_10:.3f} |  Val. Acc-Adam_10: {valid_acc_Adam_10*100:.2f}%')

# Adam_10
model_RNN.load_state_dict(torch.load('tut1-model_Adam_10.pt'))
test_loss_Adam_10, test_acc_Adam_10 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adam_10: {test_loss_Adam_10:.3f} | Test Acc_Adam_10: {test_acc_Adam_10*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss-Adam_10: 0.684 | Train Acc-Adam_10: 55.86%
	 Val. Loss-Adam_10: 0.663 |  Val. Acc-Adam_10: 59.87%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss-Adam_10: 0.667 | Train Acc-Adam_10: 58.35%
	 Val. Loss-Adam_10: 0.677 |  Val. Acc-Adam_10: 55.65%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-Adam_10: 0.656 | Train Acc-Adam_10: 60.63%
	 Val. Loss-Adam_10: 0.658 |  Val. Acc-Adam_10: 60.32%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-Adam_10: 0.607 | Train Acc-Adam_10: 66.82%
	 Val. Loss-Adam_10: 0.661 |  Val. Acc-Adam_10: 59.52%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adam_10: 0.604 | Train Acc-Adam_10: 65.19%
	 Val. Loss-Adam_10: 0.695 |  Val. Acc-Adam_10: 53.05%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-Adam_10: 0.679 | Train Acc-Adam_10: 56.03%
	 Val. Loss-Adam_10: 0.681 |  Val. Acc-Adam_10: 55.04%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss-Adam_10: 0.661 | Train Acc-Adam_10: 58.85%
	 Val. Loss-Adam_10: 0.664 |  Val. Acc-Adam_10: 58.77%
Epoch: 08 | Epoch Ti

<ipython-input-35-306daa038925>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adam_10.pt'))


Test Loss_Adam_10: 0.600 | Test Acc_Adam_10: 67.78%


In [36]:
## Adam_50
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model_RNN_main = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task
model_RNN = model_RNN_main.to(device)
criterion = criterion.to(device)

best_valid_loss_Adam_50 = float('inf')
epoch=0
N_EPOCHS=50
optimizer_Adam = optim.Adam(model_RNN_main.parameters(), lr=1e-3) # Adam optimizer - Stochastic gradient descent
for epoch in range(N_EPOCHS):
    start_time_Adam_50 = time.time()

    train_loss_Adam_50, train_acc_Adam_50 = train(model_RNN, train_iterator, optimizer_Adam, criterion)
    valid_loss_Adam_50, valid_acc_Adam_50 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adam_50 = time.time()

    epoch_mins_Adam_50, epoch_secs_Adam_50 = epoch_time(start_time_Adam_50, end_time_Adam_50)

    if valid_loss_Adam_50 < best_valid_loss_Adam_50:
        best_valid_loss = valid_loss_Adam_50
        torch.save(model_RNN.state_dict(), 'tut1-model_Adam_50.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adam_50}m {epoch_secs_Adam_50}s')
    print(f'\tTrain Loss-Adam_50: {train_loss_Adam_50:.3f} | Train Acc-Adam_50: {train_acc_Adam_50*100:.2f}%')
    print(f'\t Val. Loss-Adam_50: {valid_loss_Adam_50:.3f} |  Val. Acc-Adam_50: {valid_acc_Adam_50*100:.2f}%')

# Adam_50
model_RNN.load_state_dict(torch.load('tut1-model_Adam_50.pt'))
test_loss_Adam_50, test_acc_Adam_50 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adam_50: {test_loss_Adam_50:.3f} | Test Acc_Adam_50: {test_acc_Adam_50*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss-Adam_50: 0.689 | Train Acc-Adam_50: 54.43%
	 Val. Loss-Adam_50: 0.680 |  Val. Acc-Adam_50: 55.56%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss-Adam_50: 0.693 | Train Acc-Adam_50: 52.12%
	 Val. Loss-Adam_50: 0.688 |  Val. Acc-Adam_50: 53.18%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-Adam_50: 0.689 | Train Acc-Adam_50: 53.84%
	 Val. Loss-Adam_50: 0.692 |  Val. Acc-Adam_50: 52.83%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss-Adam_50: 0.678 | Train Acc-Adam_50: 56.42%
	 Val. Loss-Adam_50: 0.673 |  Val. Acc-Adam_50: 57.03%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adam_50: 0.654 | Train Acc-Adam_50: 60.36%
	 Val. Loss-Adam_50: 0.664 |  Val. Acc-Adam_50: 59.04%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-Adam_50: 0.626 | Train Acc-Adam_50: 64.98%
	 Val. Loss-Adam_50: 0.671 |  Val. Acc-Adam_50: 59.47%
Epoch: 07 | Epoch Time: 0m 4s
	Train Loss-Adam_50: 0.587 | Train Acc-Adam_50: 68.30%
	 Val. Loss-Adam_50: 0.659 |  Val. Acc-Adam_50: 60.51%
Epoch: 08 | Epoch Ti

<ipython-input-36-618cde435688>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adam_50.pt'))


Test Loss_Adam_50: 0.936 | Test Acc_Adam_50: 68.59%


In [6]:
# Model Settings - 4
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

# 定义模型
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dims, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        layers = []
        in_dim = embedding_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.ReLU())
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, output_dim))
        self.fc = nn.Sequential(*layers)

    def forward(self, text, text_lengths=None):
        embedded = self.embedding(text).mean(dim=0)  # 平均池化
        return self.fc(embedded)

class CNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim, kernel_sizes, num_filters):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim) # 随机初始化嵌入
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (ks, embedding_dim)) for ks in kernel_sizes
        ])
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_dim)

    #def forward(self, text, text_lengths=None):
    #    embedded = self.embedding(text).unsqueeze(1)  # 添加通道维度
    #    conved = [torch.relu(conv(embedded)).squeeze(3) for conv in self.convs]
    #    pooled = [torch.max(conv, dim=2)[0] for conv in conved]
    #    cat = torch.cat(pooled, dim=1)
    #    return self.fc(cat)

    def forward(self, text, text_lengths=None):
        # 1. 嵌入层：将输入转换为嵌入向量
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        embedded = embedded.unsqueeze(1)  # 添加通道维度 [batch_size, 1, seq_len, embedding_dim]
        print(f"Embedded shape: {embedded.shape}")  # 调试输出形状

        # 2. 卷积 + ReLU
        conved = [torch.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        for i, conv_out in enumerate(conved):
            print(f"Conv {i+1} output shape: {conv_out.shape}")  # 调试每个卷积输出

        # 3. 最大池化
        pooled = [torch.max(conv, dim=2)[0] for conv in conved]
        for i, pool_out in enumerate(pooled):
            print(f"Pooled {i+1} output shape: {pool_out.shape}")  # 调试池化输出

        # 4. 特征拼接
        cat = torch.cat(pooled, dim=1)  # [batch_size, len(kernel_sizes) * num_filters]
        print(f"Concatenated shape: {cat.shape}")  # 调试拼接后形状

        # 5. 全连接层
        output = self.fc(cat)  # [batch_size, output_dim]
        print(f"Output shape: {output.shape}")  # 调试最终输出形状

        return output

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, _) = self.lstm(packed_embedded)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1) if self.lstm.bidirectional else hidden[-1]
        return self.fc(hidden)

# 定义训练和验证函数
def run_experiment(model, optimizer, criterion, train_iterator, valid_iterator, test_iterator, model_name):
    best_valid_loss = float('inf')
    for epoch in range(50):
        start_time = time.time()
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'{model_name}.pt')
        print(f'Epoch: {epoch+1:02} | {model_name} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    # 测试模型
    model.load_state_dict(torch.load(f'{model_name}.pt'))
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    print(f'{model_name} Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# 实验设置
models = {
    "OneLayerFFNN": FeedForwardNN(INPUT_DIM, EMBEDDING_DIM, [500], OUTPUT_DIM),
    "TwoLayerFFNN": FeedForwardNN(INPUT_DIM, EMBEDDING_DIM, [500, 300], OUTPUT_DIM),
    "ThreeLayerFFNN": FeedForwardNN(INPUT_DIM, EMBEDDING_DIM, [500, 300, 200], OUTPUT_DIM),
    "CNN": CNN(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, [1, 2, 3], 100),
    "LSTM": LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, bidirectional=False),
    "BiLSTM": LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, bidirectional=True)
}

for model_name, model in models.items():
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    run_experiment(model, optimizer, criterion, train_iterator, valid_iterator, test_iterator, model_name)

Epoch: 01 | OneLayerFFNN | Time: 0m 4s
	Train Loss: 0.553 | Train Acc: 70.93%
	 Val. Loss: 0.441 |  Val. Acc: 80.75%
Epoch: 02 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.367 | Train Acc: 84.54%
	 Val. Loss: 0.342 |  Val. Acc: 85.54%
Epoch: 03 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.263 | Train Acc: 89.47%
	 Val. Loss: 0.335 |  Val. Acc: 87.29%
Epoch: 04 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.224 | Train Acc: 92.07%
	 Val. Loss: 0.313 |  Val. Acc: 87.76%
Epoch: 05 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.165 | Train Acc: 94.17%
	 Val. Loss: 0.302 |  Val. Acc: 88.68%
Epoch: 06 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.145 | Train Acc: 95.45%
	 Val. Loss: 0.317 |  Val. Acc: 88.26%
Epoch: 07 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.115 | Train Acc: 96.69%
	 Val. Loss: 0.318 |  Val. Acc: 88.71%
Epoch: 08 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.073 | Train Acc: 97.87%
	 Val. Loss: 0.384 |  Val. Acc: 87.78%
Epoch: 09 | OneLayerFFNN | Time: 0m 2s
	Train Loss: 0.077 | Trai

<ipython-input-6-09a83216f5a7>:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{model_name}.pt'))


OneLayerFFNN Test Loss: 0.358 | Test Acc: 85.77%
Epoch: 01 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.535 | Train Acc: 72.43%
	 Val. Loss: 0.444 |  Val. Acc: 79.86%
Epoch: 02 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.371 | Train Acc: 84.37%
	 Val. Loss: 0.341 |  Val. Acc: 85.77%
Epoch: 03 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.258 | Train Acc: 89.87%
	 Val. Loss: 0.321 |  Val. Acc: 87.01%
Epoch: 04 | TwoLayerFFNN | Time: 0m 3s
	Train Loss: 0.201 | Train Acc: 92.59%
	 Val. Loss: 0.320 |  Val. Acc: 87.77%
Epoch: 05 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.136 | Train Acc: 95.30%
	 Val. Loss: 0.329 |  Val. Acc: 88.17%
Epoch: 06 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.110 | Train Acc: 96.26%
	 Val. Loss: 0.414 |  Val. Acc: 87.76%
Epoch: 07 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.064 | Train Acc: 98.00%
	 Val. Loss: 0.428 |  Val. Acc: 87.79%
Epoch: 08 | TwoLayerFFNN | Time: 0m 2s
	Train Loss: 0.091 | Train Acc: 97.59%
	 Val. Loss: 0.502 |  Val. Acc: 87.74%
Epoch: 09 | Two

ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([545]))